In [16]:
import pandas as pd
import re
import nltk
from nltk.stem.isri import ISRIStemmer
from tqdm import tqdm
from arabert.preprocess import ArabertPreprocessor
from transformers import BertTokenizer, BertModel
import torch

  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)


In [2]:
train = pd.read_csv('dataset/train.csv')

In [3]:
train['stance'] += 1
train['stance'].value_counts()

2    5538
1    1012
0     438
Name: stance, dtype: int64

In [4]:
# Down sample class 1 to be 2500
train_2 = train[train['stance'] == 2].sample(n=2500, random_state=42, replace=True)
train_1 = train[train['stance'] == 1]
train_0 = train[train['stance'] == 0]

# Upsample classes 1 & 0 to be 2500
train_1 = train_1.sample(n=2500, replace=True, random_state=42)
train_0 = train_0.sample(n=2500, replace=True, random_state=42)

train = pd.concat([train_0, train_1, train_2])


In [5]:
# Preprocessing using arabert
model_name = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)
# Can convert words to ids using: tokenizer.convert_tokens_to_ids()

def preprocess_arabert(text, embedding=True):
    """
    This function preprocesses the text using arabert.
    It's essentially a full pipeline that even returns the word embeddings.

    Parameters
    ----------
    text: str
        The text to be preprocessed
    embedding: bool
        Whether to return the word embeddings or not
    
    Returns
    -------
    output: list
        The preprocessed text
    """
    def clean_text(text):
        # remove any word with + in it
        text = re.sub(r'\S*\+\S*', '', text)
        # remove non arabic characters
        text = re.sub(r'[^\u0600-\u06FF]', ' ', text)
        # remove extra spaces
        return text
    output = arabert_prep.preprocess(text)
    output = clean_text(output)
    tokenized = tokenizer.tokenize(output)
    # pad all tokenized sentences to be the same length
    max_len = 0
    if embedding:
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized)
        tokens_tensor = torch.tensor([indexed_tokens])
        with torch.no_grad():
            outputs = bert_model(tokens_tensor)
            encoded_layers = outputs[0]
        return encoded_layers
    else:
        return tokenized

# preprocess the entire dataset
tqdm.pandas()
train_arabert = train['text'].progress_apply(preprocess_arabert)

# This is for padding so that all sentences are the same length
# I don't suggest doing this and saving the padded sentences because
# the size goes from 550 MB to more than 2.5 GB, instead do this when training
# pad_embedding = torch.zeros(1, 1, 768)
# max_len = 0
# for i in range(len(train_arabert)):
#     if train_arabert[i].shape[1] > max_len:
#         max_len = train_arabert[i].shape[1]
# for i in range(len(train_arabert)):
#     if train_arabert[i].shape[1] < max_len:
#         pad = torch.zeros(1, max_len - train_arabert[i].shape[1], 768)
#         train_arabert[i] = torch.cat((train_arabert[i], pad), dim=1)

[2022-12-13 16:52:02,926 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Downloading: 100%|██████████| 543M/543M [02:36<00:00, 3.48MB/s] 
c:\Users\swak\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swak\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us

In [6]:
train_arabert.to_pickle('dataset/train_arabert_not_padded.pkl') # save the preprocessed dataset (550 MB) - put into git ignore
# It's only here so that there's no need to do the preprocessing again when testing the code
# train_arabert = pd.read_pickle('dataset/train_arabert_not_padded.pkl')

In [7]:
# This function is made because the process of cleaning arabic text is complex
# and depends on many unicodes done in many steps
def clean_arabic(text):
    # ! to understand this nonsense you need this link open 
    """ https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AE%D8%B7_
        %D8%A7%D9%84%D8%B9%D8%B1%D8%A8%D9%8A_%D9%81%D9%8A_%D9%8A%D9
        %88%D9%86%D9%8A%D9%83%D9%88%D8%AF   """

    # remove التشكيل
    text = re.sub(r'[\u0600-\u061F]', '', text)
    text = re.sub(r'[\u064B-\u066D]', '', text)

    # Because of all the idiots that were bypassing twitters' spam filters using
    # special characters like this idiot: كو.ر.ونا We'll remove all the special 
    # before everything else

    # remove special characters
    text = re.sub(r'[\u0024-\u003F]', '', text)
    text = re.sub(r'[\u005B-\u0060]', '', text)
    text = re.sub(r'[\u007B-\u007E]', '', text)

    # replace weird characters with more standard ones
    # 1. replace چ with ج
    text = re.sub(r'چ','ج',text)

    # 2. replace ڤ ڨ with ف
    text = re.sub(r'ڤ','ف',text)
    text = re.sub(r'ڨ','ف',text)

    # 3. replace ڠ with ق
    text = re.sub(r'ڠ','غ',text)
    
    # 4. replace ٱ	ٲ	ٳ	◌ٴ	ٵ with ا
    string = ['ٱ','ٲ','ٳ','ٴ','ٵ', 'آ', 'أ', 'إ']
    for char in string:
        text = re.sub(char,'ا',text)

    # 5. replace ٶ	ٷ with و
    string = ['ٶ','ٷ']
    for char in string:
        text = re.sub(char,'و',text)

    # 6. replace ٸ ی with ي
    text = re.sub(r'ٸ','ي',text) 
    text = re.sub(r'ی','ي',text)
    
    # 7. replace پ	with ب
    text = re.sub(r'پ','ب',text)

    # 8. replace ژ with ز
    text = re.sub(r'ژ','ز',text)

    # 9. replace ک ڪ ګ ڬ ڭ ڮ گ ڰ ڱ ڲ ڳ ڴ with ك
    string = ['ک', 'ڪ', 'ګ', 'ڬ', 'ڭ', 'ڮ', 'گ', 'ڰ', 'ڱ', 'ڲ', 'ڳ', 'ڴ']
    for char in string:
        text = re.sub(char,'ك',text)
    # 10. replace ھ with ه
    text = re.sub(r'ھ','ه',text)

    # remove all extra arabic characters (shift + ت) 
    text = re.sub(r'ـ','',text)

    # remove non arabic characters
    text = re.sub(r'[^\u0620-\u064A\s]',' ',text)
    
    return text

In [8]:
def clean_text(text):
    " removes all non arabic characters & replaces all spaces with a single space "
    # TODO: figure out how this works
    # remove all non arabic characters & numbers
    # text = re.sub(r'[^\u0621-\u064A\u0660-\u0669\s]','',text)
    
    # remove all words with # in them
    text = re.sub(r'[^\s]*#[^\s]*',' ',text)
    
    # arabic letters clean up 
    text = clean_arabic(text)
        
    # replace all white spaces with a single space
    text = re.sub(r'\s+',' ',text)
    
    return text 

In [9]:
cleaned_train_data = train['text'].apply(lambda x: clean_text(x))

In [10]:
# Might need to download punkt & stopwords
# ! Run the following lines in an empty cell if the code doesn't work
# nltk.download('punkt')
# nltk.download('stopwords')

# currently this line just tokenizes the text using a space (I think)
tokenized_train_data = [nltk.word_tokenize(text) for text in cleaned_train_data]

In [11]:
train['text'][3185]

3185    الموت أرحم عندي من انو  آخد لقاح موهوب من دولة...
3185    الموت أرحم عندي من انو  آخد لقاح موهوب من دولة...
3185    الموت أرحم عندي من انو  آخد لقاح موهوب من دولة...
Name: text, dtype: object

In [12]:
# remove stopwords
stopwords = nltk.corpus.stopwords.words('arabic')
tokenized_no_stopwords_train_data = [[word for word in text if word not in stopwords] for text in tokenized_train_data]

# Here we're looking for more stopwords that are 2 characters or less
# we spend hours doing just this for two or three character words
stopwords_extracted_from_dataset = ['ال', 'اي', 'ان', 'تم', 'بن', 
                                    'او', 'اي', 'عم', 'ام', 'رض',
                                    'في', 'فى', 'رب', 'سم', 'خط',
                                    'ول', 'زي', 'دي', 'اذ', 'ده',
                                    'دى', 'انه', 'ابو', 'احد']

tokenized_no_stopwords_train_data_v2 = [[word for word in text if word not in stopwords_extracted_from_dataset] for text in tokenized_no_stopwords_train_data]

In [13]:
stopwords_set = []
for text in tokenized_no_stopwords_train_data_v2:
    for word in text:
        if len(word) <= 2:
            stopwords_set.append((word, str(tokenized_no_stopwords_train_data_v2.index(text))))

stopwords_set.sort()

In [17]:
# Lemmatization
stemmed_train = []

pd.to_csv('stemmed_train.csv')


for text in tqdm(tokenized_no_stopwords_train_data_v2):
    curr_list = []
    for word in text:
        curr_list.append(ISRIStemmer().stem(word))
    stemmed_train.append(curr_list)

# stemmed_train = [[stemmer.stem(word) for word in text] for text in tokenized_no_stopwords_train_data_v2]

AttributeError: module 'pandas' has no attribute 'to_csv'

In [ ]:
# Write all of the texts into a file for visual comparison
with open('output/original_train.txt', 'w') as f:
    for text in train['text']:
        f.write(text + '\n\n')
f.close()

with open('output/cleaned_up_train.txt', 'w') as f:
    for text in cleaned_train_data:
        f.write(text + '\n\n')
f.close()

with open('output/tokens_train.txt', 'w') as f:
    for text in tokenized_train_data:
        f.write(str(text) + '\n\n')
f.close()

with open('output/tokens_no_stopwords_train.txt', 'w') as f:
    for text in tokenized_no_stopwords_train_data:
        f.write(str(text) + '\n\n')
f.close()

with open('output/words_less_than_2.txt', 'w') as f:
    for word in stopwords_set:
        f.write(str(word) + '\n')
f.close()

with open('output/stemmed_train.txt', 'w') as f:
    for text in stemmed_train:
        f.write(str(text) + '\n\n')
f.close()
# use alt + z to toggle word wrap in vscode 